complex spikes overlapping, model seen samples in training set, accuracy not accurate. 

In [6]:
import os
import numpy as np
import matplotlib.pyplot as plt

# os.chdir('/Users/xiao-zy19/Desktop/good_cs/Zhenyu')

data_buf = np.load('data.npy')
index_buf = np.load('index.npy')
data = data_buf[:1055000]
index = index_buf[index_buf < 1055000].astype(int)
print(data.shape)
print(index.shape)

(64, 6122790)
(37,)


Sliding Window: try 20ms length (600 samples) and 5ms step (150 samples) for each window?

In [7]:
# define a sliding window
def sliding_window(data, window_size=600, stride=150):
    window_data = np.zeros(((len(data)-window_size)//stride, window_size))
    index = np.zeros(((len(data)-window_size)//stride, window_size))
    for row in range((len(data)-window_size)//stride):
        window_data[row, :] = data[row*stride:row*stride+window_size]
        index[row, :] = np.arange(row*stride, row*stride+window_size) 
    return window_data, index

windowed_data, windowed_index = sliding_window(data[53, :], window_size=600, stride=150)

label = np.zeros(windowed_data.shape[0])
for i in range(windowed_data.shape[0]):
    if np.any(np.isin(windowed_index[i, :], index)):
        label[i] = 1
    else:
        label[i] = 0

In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=7, padding='same')
        self.bn1 = nn.BatchNorm1d(num_features=16)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=5, padding='same')
        self.bn2 = nn.BatchNorm1d(num_features=32)
        self.conv3 = nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3, padding='same')
        self.bn3 = nn.BatchNorm1d(num_features=64)
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        # fc
        self.fc1 = nn.Linear(64, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        x = self.global_avg_pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        
        return x 

In [9]:
# emmm, similar data in train set and the test set？
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch

data_tensor = torch.tensor(windowed_data[:, None, :], dtype=torch.float32) 
label_tensor = torch.tensor(label, dtype=torch.float32).unsqueeze(1)

# balance the dataset
positive_index = np.where(label == 1)[0]
negative_index = np.where(label == 0)[0]
np.random.seed(42)
negative_index = np.random.choice(negative_index, size=positive_index.shape[0], replace=False)
data_tensor = torch.cat((data_tensor[positive_index], data_tensor[negative_index]), dim=0)
label_tensor = torch.cat((label_tensor[positive_index], label_tensor[negative_index]), dim=0)

X_train, X_test, y_train, y_test = train_test_split(data_tensor, label_tensor, test_size=0.2, random_state=42, stratify=label_tensor)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 创建训练和测试的TensorDataset
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# 创建DataLoader
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型、损失函数和优化器
model = ConvNet()
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 训练模型
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in test_loader:
        outputs = model(inputs)
        predicted = (outputs.data > 0.5).float()  # 使用0.5作为阈值来判断预测类别
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy on test set: {accuracy}%')

torch.Size([193, 1, 600]) torch.Size([49, 1, 600]) torch.Size([193, 1]) torch.Size([49, 1])
Epoch 1/10, Loss: 0.6906640529632568
Epoch 2/10, Loss: 0.7226507663726807
Epoch 3/10, Loss: 0.7262158989906311
Epoch 4/10, Loss: 0.6756369471549988
Epoch 5/10, Loss: 0.6277449727058411
Epoch 6/10, Loss: 0.9014409780502319
Epoch 7/10, Loss: 0.5537046790122986
Epoch 8/10, Loss: 0.5201243162155151
Epoch 9/10, Loss: 0.4180636703968048
Epoch 10/10, Loss: 1.164966344833374
Accuracy on test set: 100.0%


In [10]:
# import matplotlib.pyplot as plt

# plt.figure()
# for i in range(len(index)):
#     plt.plot(np.arange(0, 600), data[53, index[i]-200:index[i]+400])
# plt.show()


# def a cnn1d model to extract features from windowed data
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

# class CNN1D(nn.Module):
#     def __init__(self):
#         super(CNN1D, self).__init__()
#         self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1)
#         self.pool1 = nn.MaxPool1d(kernel_size=2, stride=2)
#         self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1)
#         self.pool2 = nn.MaxPool1d(kernel_size=2, stride=2)
#         self.fc1 = nn.Linear(32*150, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 1)
        
#     def forward(self, x):
#         x = F.relu(self.conv1(x))
#         x = self.pool1(x)
#         x = F.relu(self.conv2(x))
#         x = self.pool2(x)
#         x = x.view(-1, 32*150)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x